In [0]:
import numpy as np
import pandas as pd
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

In [0]:
ext_loc_url = w.external_locations.get("emhollanding").url

Trasnactional data for products sold in stores, simulated IoT stream.

Attributes: 
```
id: unique transaction id
product_id: unique product id
customer_id: unique customer id
store_id: unique store id
receipt_id: unique receipt id, collection of products in a store done by a customer at one time
transaction_date: date of transaction
transaction_amount: price per product
```

Each receipt is dumped as a json to blob. A receipt can contain one or multiple transactions

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, rand, expr
import uuid
from datetime import datetime, timedelta
import random

# Function to generate random date
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 3, 10)
def random_date():
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Generate 50,000 rows
data = [
    (
        str(uuid.uuid4()),  # transaction_id
        random.randint(1, 150),  # product_id
        random.randint(1, 100),  # customer_id
        random.randint(1, 10),  # store_id
        random.randint(1000, 9999),  # receipt_id
        random_date().strftime("%Y-%m-%d"),  # transaction_date
        round(random.uniform(5, 500), 2)  # transaction_amount
    )
    for _ in range(50000)
]

# Define schema and create DataFrame
columns = ["id", "product_id", "customer_id", "store_id", "receipt_id", "transaction_date", "transaction_amount"]
df = spark.createDataFrame(data, columns)


In [0]:
df.display()

In [0]:
print(df.select("product_id").distinct().count(),
df.select("customer_id").distinct().count(),
df.select("store_id").distinct().count(),
df.select("receipt_id").distinct().count())

In [0]:
df.groupBy("receipt_id").count().display()

In [0]:
df.write.mode("overwrite").format("json").option("partitionBy", "transaction_date").save(f"{ext_loc_url}/bootcamp/iot_stream/")

Create a product dimension with attributes:
```
id: unique product id
product_name: Product name
product_description: Product description
date: date when added
```

In [0]:
# Sample product categories and descriptions
product_categories = [
    ("Smartphone", "Latest model with high-resolution display and fast processor"),
    ("Laptop", "Lightweight and powerful laptop for professionals"),
    ("Wireless Earbuds", "Noise-canceling earbuds with long battery life"),
    ("Smartwatch", "Fitness tracking smartwatch with heart rate monitor"),
    ("Blender", "High-speed blender perfect for smoothies and soups"),
    ("Coffee Maker", "Programmable coffee maker with built-in grinder"),
    ("Vacuum Cleaner", "Cordless vacuum cleaner with powerful suction"),
    ("Air Fryer", "Oil-free cooking with crispy results"),
    ("Gaming Console", "Next-gen console with stunning graphics"),
    ("Desk Chair", "Ergonomic office chair with lumbar support"),
    ("LED TV", "Ultra HD smart TV with vibrant colors"),
    ("Wireless Speaker", "Portable speaker with deep bass and 360° sound"),
    ("Tablet", "Compact tablet for entertainment and productivity"),
    ("Running Shoes", "Lightweight and breathable running shoes"),
    ("Backpack", "Water-resistant backpack with multiple compartments"),
    ("Sunglasses", "Polarized sunglasses with UV protection"),
    ("Yoga Mat", "Non-slip yoga mat with extra cushioning"),
    ("Cookware Set", "Durable non-stick cookware set for all your needs"),
    ("Hair Dryer", "Fast-drying hair dryer with adjustable heat settings"),
    ("Electric Toothbrush", "Smart toothbrush with multiple brushing modes")
]

# Function to generate random date
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 3, 10)
def random_date():
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Generate 150 products with variations
data = []
for i in range(150):
    category, description = random.choice(product_categories)
    product_name = f"{category} Model {random.randint(100, 999)}"
    data.append((
        i,  # unique product_id
        product_name,
        category,
        f"{description}. Model {random.randint(1, 10)} with enhanced features.",
        random_date().strftime("%Y-%m-%d")  # date added
    ))

data += data
# Define schema and create DataFrame
columns = ["id", "product_name", "product_category","product_description", "date"]
df_prod = spark.createDataFrame(data, columns)

In [0]:
df_prod.display()

In [0]:
df_prod.write.mode("overwrite").format("json").option("partitionBy", "date").save(f"{ext_loc_url}/bootcamp/products/")

Create a customer dimension with attributes:
```
id: unique customer id
customer_name: Customer name
gender: Customer gender
address: Customer address
date: date when added
```

In [0]:
# Sample customer names and addresses
male_customers = [
    "Bob Smith", "Charlie Davis", "Ethan Wright", "George Harris", "Ian Clark",
    "Kevin Hall", "Michael Scott", "Oscar Turner", "Samuel Rogers", "Thomas Alexander",
    "Walter Murphy", "Xavier Reed", "Zachary Barnes", "Brian Gibson", "David Simmons",
    "Franklin Mitchell", "Henry Sanders", "Jack Coleman", "Liam Morris", "Noah Edwards",
    "Peter Clark", "Robert Jenkins", "Vincent Hayes", "William Parker", "Yusuf Holmes",
    "Adrian Fletcher", "Caleb Douglas", "Elliot Cooper", "Gordon Stewart", "Isaac Baldwin",
    "Kyle Armstrong", "Mason Barrett", "Owen Simpson", "Patrick Weaver", "Quincy Bennett",
    "Riley Hayes", "Travis Dunn", "Victor Patterson"
]

female_customers = [
    "Alice Johnson", "Diana Evans", "Fiona Green", "Hannah Lewis", "Julia Baker",
    "Laura Adams", "Nina Brooks", "Paula Carter", "Rachel Hughes", "Tina Perry",
    "Victoria Howard", "Yasmine Cooper", "Amy Fisher", "Catherine Nelson", "Emma Ward",
    "Grace Elliott", "Isla Bryant", "Kylie Ross", "Mia Powell", "Olivia Hughes",
    "Queenie Stewart", "Sophia Foster", "Talia Murray", "Vera Gardner", "Willow Bennett",
    "Xena Baldwin", "Zoe Carr", "Bethany Stone", "Daisy Franklin", "Faith Matthews",
    "Hailey Clarke", "Jasmine Reid", "Lydia Fox", "Madeline Curtis", "Natalie Owens",
    "Penelope Carter", "Savannah Mitchell", "Ursula Lambert"
]

addresses = [
    "123 Elm St, Springfield", "456 Oak Ave, Riverside", "789 Pine Rd, Lincoln",
    "101 Maple Blvd, Franklin", "202 Cedar Ln, Shelbyville", "303 Birch St, Madison",
    "404 Walnut Dr, Auburn", "505 Aspen Ct, Brookfield", "606 Cherry Cir, Lexington",
    "707 Magnolia Pl, Hamilton"
]

genders = ["Male", "Female", "Non-Binary"]

# Function to generate random date
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 3, 10)
def random_date():
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

data = []
# Generate 100 customers
for i in range(50):
    male_name = random.choice(male_customers)
    female_name = random.choice(female_customers)
    data.append((
        i*2+1, male_name, "Male", random.choice(addresses), random.randint(18, 100), random_date().strftime("%Y-%m-%d")
    ))
    data.append((
        i*2+2, female_name, "Female", random.choice(addresses), random.randint(18, 100), random_date().strftime("%Y-%m-%d")
    ))

data += data
# Define schema and create DataFrame
columns = ["id", "customer_name", "gender", "address", "age", "date"]
df_cust = spark.createDataFrame(data, columns)

In [0]:
df_cust.display()

In [0]:
df_cust.write.mode("overwrite").format("json").option("partitionBy", "date").save(f"{ext_loc_url}/bootcamp/customers/")

Create a store dimension with attributes:
```
id: unique store id
store_name: Store name
address: Store address
city: City location
date: date when added
```

In [0]:
# Sample store names and addresses
store_names = [
    "SuperMart", "QuickShop", "MegaStore", "ValueMart", "ShopEase",
    "DailyNeeds", "ExpressMart", "BudgetBazaar", "HyperMart", "RetailHub"
]

addresses = [
    "101 Main St", "202 Oak Ave", "303 Pine Rd", "404 Maple Blvd", "505 Cedar Ln",
    "606 Birch St", "707 Walnut Dr", "808 Aspen Ct", "909 Cherry Cir", "100 Elm St"
]

cities = [
    "Springfield", "Riverside", "Lincoln", "Franklin", "Shelbyville",
    "Madison", "Auburn", "Brookfield", "Lexington", "Hamilton"
]

# Function to generate random date
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 3, 10)
def random_date():
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Generate 10 stores
data = [
    (
        i+1,  # unique store_id
        store_names[i],
        addresses[i],
        cities[i],
        random_date().strftime("%Y-%m-%d")  # date added
    )
    for i in range(10)
]
data += data
# Define schema and create DataFrame
columns = ["id", "store_name", "address", "city", "date"]
df_store = spark.createDataFrame(data, columns)

In [0]:
df_store.display()

In [0]:
df_store.write.mode("overwrite").format("json").option("partitionBy", "date").save(f"{ext_loc_url}/bootcamp/stores/")